In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras import initializers

import pandas as pd
import numpy as np
import keras
import os
import joblib

Using TensorFlow backend.
/home/neo/MS/Projects/KerasDeepSpeech/env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class LossHistory(keras.callbacks.Callback):
    def __init__(self):
        self.losses = np.ones((1,1))
        
    def on_train_begin(self, logs={}):
        pass

    def on_batch_end(self, batch, logs={}):
        self.losses = np.vstack((self.losses, logs.get('loss')))

In [3]:
history = LossHistory()
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph/timit_nodbn', histogram_freq=1, write_graph=True, write_images=True)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=5, \
                          verbose=1, mode='auto')
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)

In [4]:
X = joblib.load(open("./features/features_float32.pkl", "rb"))
Y = joblib.load(open("./features/labels_int16_mapped.pkl", "rb"))
X -=  X.mean(axis=0)
X /= X.std(axis=0)
# Y = keras.utils.to_categorical(Y, num_classes=1944)
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.3, random_state=0)

In [5]:
wts = joblib.load("./features/kaldi_dbn_weights.pkl")
w1 = [np.asanyarray(wts[:1024], dtype=np.float32).T, np.asanyarray(wts[1024], dtype=np.float32)]
w2 = [np.asanyarray(wts[1025:2049], dtype=np.float32).T, np.asanyarray(wts[2049], dtype=np.float32)]
w3 = [np.asanyarray(wts[2050:3074], dtype=np.float32).T, np.asanyarray(wts[3074], dtype=np.float32)]
w4 = [np.asanyarray(wts[3075:4099], dtype=np.float32).T, np.asanyarray(wts[4099], dtype=np.float32)]
w5 = [np.asanyarray(wts[4100:5124], dtype=np.float32).T, np.asanyarray(wts[5124], dtype=np.float32)]
w6 = [np.asanyarray(wts[5125:6149], dtype=np.float32).T, np.asanyarray(wts[6149], dtype=np.float32)]

In [6]:
def get_model(dbn=False):
    if(dbn):
        model = Sequential([
        Dense(units=1024, activation='relu', input_dim=440, weights=w1),
        Dropout(0.2),
        BatchNormalization(),

        Dense(1024, activation='relu', weights=w6),
        Dropout(0.2),
        BatchNormalization(),

        Dense(1024, activation='relu', weights=w5),
        Dropout(0.2),
        BatchNormalization(),

        Dense(1024, activation='relu', weights=w4),
        Dropout(0.2),
        BatchNormalization(),

        Dense(1024, activation='relu', weights=w3),
        Dropout(0.2),
        BatchNormalization(),

        Dense(1024, activation='relu', weights=w2),
        Dropout(0.2),
        BatchNormalization(),

        Dense(units=48, activation='softmax'),
        ])
    else:
        model = Sequential([
        Dense(units=1024, activation='relu', input_dim=440),
        Dropout(0.2),
        BatchNormalization(),

        Dense(1024, activation='relu'),
        Dropout(0.2),
        BatchNormalization(),

        Dense(1024, activation='relu'),
        Dropout(0.2),
        BatchNormalization(),

        Dense(1024, activation='relu'),
        Dropout(0.2),
        BatchNormalization(),

        Dense(1024, activation='relu'),
        Dropout(0.2),
        BatchNormalization(),

        Dense(1024, activation='relu'),
        Dropout(0.2),
        BatchNormalization(),

        Dense(units=48, activation='softmax'),
        ])
    return model

In [7]:
model = get_model(dbn=False)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              451584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              1049600   
__________

In [9]:
validation_split = 0.1
validation_size = int(len(X_train)*validation_split)
batch_size=256

In [10]:
model.fit(X_train, y_train, epochs=50, batch_size=256,validation_split=0.1,
             callbacks=[tbCallBack, early_stopping, reduce_lr])

Train on 708638 samples, validate on 78738 samples
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
INFO:tensorflow:Summary name batch_normalization_1/gamma:0 is illegal; using batch_normalization_1/gamma_0 instead.
INFO:tensorflow:Summary name batch_normalization_1/gamma:0 is illegal; using batch_normalization_1/gamma_0 instead.
INFO:tensorflow:Summary name batch_normalization_1/beta:0 is illegal; using batch_normalization_1/beta_0 instead.
INFO:tensorflow:Summary name batch_normalization_1/beta:0 is illegal; using batch_normalization_1/beta_0 instead.
INFO:tensorflow:Summary name batch_normalization_1/moving_mean:0 is illegal; using batch_normalization_1/moving_mean_0 instead.
INFO:tens

INFO:tensorflow:Summary name dense_7/bias:0 is illegal; using dense_7/bias_0 instead.
INFO:tensorflow:Summary name dense_7/bias:0 is illegal; using dense_7/bias_0 instead.
Epoch 1/50
708638/708638 [==============================] - 62s - loss: 1.1723 - acc: 0.6387 - val_loss: 0.8923 - val_acc: 0.7107
Epoch 2/50
708638/708638 [==============================] - 56s - loss: 0.9314 - acc: 0.7012 - val_loss: 0.7852 - val_acc: 0.7429
Epoch 3/50
708638/708638 [==============================] - 52s - loss: 0.8492 - acc: 0.7244 - val_loss: 0.7355 - val_acc: 0.7561
Epoch 4/50
708638/708638 [==============================] - 54s - loss: 0.7889 - acc: 0.7414 - val_loss: 0.6890 - val_acc: 0.7686
Epoch 5/50
708638/708638 [==============================] - 58s - loss: 0.7432 - acc: 0.7547 - val_loss: 0.6618 - val_acc: 0.7768
Epoch 6/50
708638/708638 [==============================] - 59s - loss: 0.7074 - acc: 0.7649 - val_loss: 0.6342 - val_acc: 0.7864
Epoch 7/50
708638/708638 [======================

In [11]:
model.save('timit_nodbn_init.h5')

In [12]:
train_acc = model.evaluate(X_train, y_train)
print("Train accuracy:", train_acc)

786560/787376 [============================>.] - ETA: 0sTrain accuracy: [0.1623557306743904, 0.9501737416431286]


In [13]:
loss_and_metrics = model.evaluate(X_test, y_test)
print("Test accuracy:",loss_and_metrics)

336576/337447 [============================>.] - ETA: 0sTest accuracy: [0.42315098551744634, 0.8579747338102268]


In [14]:
del model

In [15]:
exit()

In [16]:
quit()